In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. Setup Project Path
import os
import sys

# UPDATE THIS PATH to match where you uploaded the project in your Drive
PROJECT_ROOT = '/content/drive/MyDrive/headway-prediction'

os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print(f"Current Working Directory: {os.getcwd()}")

In [ ]:
# 3. Install Dependencies
!pip install -r requirements.txt

In [ ]:
# 4. Import Libraries
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from src.config import Config
from src.data.dataset import SubwayDataGenerator
from src.models.st_covnet import HeadwayConvLSTM
from src.training.trainer import Trainer
from src.evaluator import Evaluator

print(f"Tensorflow Version:{tf.__version__}")
print(f"Gpu Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# 5. Initialize Config & Data
config = Config()
config.DATA_DIR = os.path.join(PROJECT_ROOT, 'data')

# Increase batch size for Colab A100/T4
config.BATCH_SIZE = 256 
config.EPOCHS = 30

data_gen = SubwayDataGenerator(config)
data_gen.load_data()

In [ ]:
# 6. Create Splits
total_samples = len(data_gen.headway_data)
train_end = int(total_samples * 0.7)
val_end = int(total_samples * 0.9)

train_ds = data_gen.make_dataset(start_index=0, end_index=train_end)
val_ds = data_gen.make_dataset(start_index=train_end, end_index=val_end)
test_ds = data_gen.make_dataset(start_index=val_end, end_index=total_samples)

In [ ]:
# 7. Build & Train
st_net = HeadwayConvLSTM(config)
model = st_net.build_model()
model.summary()

trainer = Trainer(model, config)
trainer.compile_model()
history = trainer.fit(train_ds, val_ds)

In [ ]:
# 8. Evaluate
evaluator = Evaluator(config)
evaluator.plot_loss(history)
evaluator.plot_spatiotemporal_prediction(model, test_ds, sample_idx=0)